# Code Gen

In [1]:
from agential.cog.prompts.agents.react import REACT_INSTRUCTION_HUMANEVAL
from agential.cog.prompts.benchmarks.humaneval import HUMANEVAL_FEWSHOT_EXAMPLES_POT
from agential.cog.strategies.react.code import ReActCodeStrategy
from agential.cog.agent.react import ReActAgent
import warnings
warnings.filterwarnings('ignore')

import dotenv

dotenv.load_dotenv()

from langchain_community.chat_models.openai import ChatOpenAI

llm = ChatOpenAI()

strat = ReActCodeStrategy(llm)

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `DocstoreExplorer` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0
  warn_deprecated(


In [2]:
inst = {"task_id": "HumanEval/0", "prompt": "from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    \"\"\"\n", "entry_point": "has_close_elements", "canonical_solution": "    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n", "test": "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False\n    assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True\n    assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False\n\n"}

question = inst['prompt']

tests = f"{inst['test']}\ncheck({inst['entry_point']})"

In [3]:
out = strat.generate(
    question=question,
    examples=HUMANEVAL_FEWSHOT_EXAMPLES_POT,
    prompt=REACT_INSTRUCTION_HUMANEVAL,
    additional_keys={}
)

<===========================================================>
Answer a coding question with interleaving Thought, Action, Observation steps. Thought can reason about the current question and plan the retrieval steps, and Action can be three types:
(1) Implement[<code>], which implements the function to answer the question.
(2) Test[<code>], which implements assert statement test cases to test the implemented code.
(3) Finish[<answer>], which returns the code implementation and finishes the task.
You have a maximum of 6 steps.

```python
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

```


Thought:
<===========================================================>
'To solve this problem, we need

In [4]:
out

'To solve this problem, we need to iterate through the list of numbers and compare each pair of numbers to see if their absolute difference is less than the given threshold. \n\n'

In [5]:
action_type, query = strat.generate_action(
    question=question,
    examples=HUMANEVAL_FEWSHOT_EXAMPLES_POT,
    prompt=REACT_INSTRUCTION_HUMANEVAL,
    additional_keys={}
)

<===========================================================>
Answer a coding question with interleaving Thought, Action, Observation steps. Thought can reason about the current question and plan the retrieval steps, and Action can be three types:
(1) Implement[<code>], which implements the function to answer the question.
(2) Test[<code>], which implements assert statement test cases to test the implemented code.
(3) Finish[<answer>], which returns the code implementation and finishes the task.
You have a maximum of 6 steps.

```python
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

```


Thought: To solve this problem, we need to iterate through the list of numbers and compare each pair o

In [12]:
a = 'Implement the has_close_elements function that iterates through the list and checks for the absolute difference between each pair of numbers. \n\n```python\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```\n\n\nObservation: The implementation of the has_close_elements function seems correct. Now, we need to test it with some sample test cases. \n\n\nAction: \n\nTest the implemented has_close_elements function with sample test cases.\n\n```python\ndef test_has_close_elements():\n    assert has_close_elements([1.0, 2.0, 3.0], 0.5) == False\n    assert has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) == True\n\ntest_has_close_elements()\n```\n\n\nObservation: The implemented function has passed the test cases successfully, indicating that it is working as expected.\n\n\nFinish: \n\n```python\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```'

a.split("```python", maxsplit=1)[-1].split("```")[0]

'\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n'

In [19]:
import re
action = 'Implement the has_close_elements function that iterates through the list and checks for the absolute difference between each pair of numbers. \n\n```python\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```\n\n\nObservation: The implementation of the has_close_elements function seems correct. Now, we need to test it with some sample test cases. \n\n\nAction: \n\nTest the implemented has_close_elements function with sample test cases.\n\n```python\ndef test_has_close_elements():\n    assert has_close_elements([1.0, 2.0, 3.0], 0.5) == False\n    assert has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) == True\n\ntest_has_close_elements()\n```\n\n\nObservation: The implemented function has passed the test cases successfully, indicating that it is working as expected.\n\n\nFinish: \n\n```python\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n```'
action_split = action.split("```python", maxsplit=1)
match = re.search(r'\b(Finish|Test|Implement)\b', action_split[0], re.IGNORECASE)

action_type = match.group(0).lower().capitalize() if match else ""
try:
    query = action_split[1].split("```")[0]
except:
    action_type = ""
    query = ""


print(match.group(0))

Implement


In [ ]:
action_type = "Implement"
query = 'def first_repeated_char(s):\n    char_set = set()\n    for char in s:\n        if char in char_set:\n            return char\n        else:\n            char_set.add(char)\n    return None'

obs = strat.generate_observation(
    idx=0,
    action_type=action_type,
    query=query
)

In [ ]:
obs

In [25]:
from agential.cog.prompts.agents.react import REACT_INSTRUCTION_MBPP
from agential.cog.prompts.benchmarks.mbpp import MBPP_FEWSHOT_EXAMPLES_REACT

question = "Write a python function to find the first repeated character in a given string."
tests = """assert first_repeated_char("abcabc") == "a"
assert first_repeated_char("abc") == None
assert first_repeated_char("123123") == "1\""""

agent = ReActAgent(
    llm=llm,
    mode={"code": "mbpp"}
)

out = agent.generate(
    question=question,
    examples=MBPP_FEWSHOT_EXAMPLES_REACT,
    prompt=REACT_INSTRUCTION_MBPP,
    additional_keys={"tests": tests},
    max_steps=3
)

<===========================================================>
Answer a coding question with interleaving Thought, Action, Observation steps. Thought can reason about the current question and plan the retrieval steps, and Action can be three types:
(1) Implement[code], which implements the function to answer the question.
(2) Test[code], which implements assert statement test cases to test the implemented code.
(3) Finish[answer], which returns the code implementation and finishes the task.
You have a maximum of 3 steps.

Here are some examples:
You are an expert Python programmer, and here is your task: Write a function to find the shared elements from the given two lists.
Your code should pass these tests:

assert set(similar_elements((3, 4, 5, 6), (5, 7, 4, 10))) == set((4, 5))
assert set(similar_elements((1, 2, 3, 4), (5, 4, 3, 7))) == set((3, 4))
assert set(similar_elements((11, 12, 14, 13), (17, 15, 14, 13))) == set((13, 14))

Thought 1: I need to write a function that accepts two